In [11]:
import cv2
import numpy as np
import gym
import growspace 

### Playing with GrowSpace
- actions (keystrokes) are explained in the funtion below 'key2action'

In [12]:
def key2action(key):
        if key == ord('a'):
            return 0 # move left
        elif key == ord('d'):
            return 1 # move right
        elif key == ord('s'):
            return 4 # stay in place
        elif key == ord('w'):
            return 2
        elif key == ord('x'):
            return 3
        elif key == ord('q'):
            return "stop"
        else:
            return None
        

def kill_window():
    """why twice? xd"""
    for _ in range(2):
        cv2.waitKey(1)
        cv2.destroyWindow('plant')        

In [13]:
# run this cell to visualize how the plant changes according to the keystrokes

env = gym.make('GrowSpaceEnv-Control-v0')

rewards = []
rewards_mean = []

run = True
while run:
    env.reset()
    img = env.get_observation(debug_show_scatter=True)

    cv2.imshow("plant", img)
    rewards = []
    
    for step in range(50):
        action = key2action(cv2.waitKey(-1))
        
        if action == "stop":
            run = False
            env.close()
            break
        if action is None:
            quit()

        b, t, c, f = env.step(action)
        rewards.append(t)
        print(f"reward after step #{step} {t}")
        cv2.imshow("plant", env.get_observation(debug_show_scatter=True))
    total = sum(rewards)

    rewards_mean.append(total)
    av = np.mean(rewards_mean)
    print("amount of rewards:", total)
    print('mean:', av)
    print("amount of rewards:", total)

reward after step #0 0.022311425889318497
reward after step #1 0.022541511787445477
reward after step #2 0.023409255620868763
reward after step #3 0.023409255620868763
reward after step #4 0.023409255620868763
reward after step #5 0.02614220510037188
reward after step #6 0.02915266055822456
reward after step #7 0.032688816558384154
reward after step #8 0.032688816558384154
reward after step #9 0.034190481983476254
reward after step #10 0.034190481983476254
amount of rewards: 0.3041341672816875
mean: 0.3041341672816875
amount of rewards: 0.3041341672816875


In [14]:
kill_window()

### Training PPO to make the plant growth towards the target (red dot)

In [2]:
from stable_baselines3 import PPO

/Users/ioneliabuzatu/Desktop/macos-growspace/venv-growspace/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
env = gym.make('GrowSpaceEnv-Control-v0')

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=5)

obs = env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
      obs = env.reset()

env.close()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50       |
|    ep_rew_mean     | 3.73     |
| time/              |          |
|    fps             | 74       |
|    iterations      | 1        |
|    time_elapsed    | 27       |
|    total_timesteps | 2048     |
---------------------------------


In [20]:
kill_window()